## Creating ylabel
## Deixando as principais variáveis



In [109]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook


In [110]:
file = 'Data/enem_todos_os_anos.csv'

cols = ['NU_ANO','TP_FAIXA_ETARIA', 'Escolaridade_Pai', 'Escolaridade_Mae', 
      'NPessoas', 'Renda', 'Computador', 'Celular', 'Internet',
      'TP_DEPENDENCIA_ADM_ESC',
      'TP_PRESENCA', 
      'NU_NOTA_OBJETIVA','NU_NOTA_REDACAO',
      'NU_NOTA_COMP1',  'NU_NOTA_COMP2',  'NU_NOTA_COMP3',  'NU_NOTA_COMP4',  'NU_NOTA_COMP5']


dfEnem = pd.read_csv(file, sep=',', encoding='ISO-8859-1', usecols=cols)

C:\Users\maria\AppData\Local\Temp\ipykernel_31536\3000320502.py:11: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  dfEnem = pd.read_csv(file, sep=',', encoding='ISO-8859-1', usecols=cols)


In [111]:
dfEnem = dfEnem[dfEnem['NU_ANO']>2010]
dfEnem = dfEnem[dfEnem['TP_PRESENCA']== 1]
dfEnem.dropna(inplace=True)
dfEnem['NU_ANO'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

In [112]:
dfEnem['NOTA_FINAL'] = dfEnem[['NU_NOTA_OBJETIVA','NU_NOTA_REDACAO']].mean(axis=1)
dfEnem['NOTA_FINAL']

169640    665.0125
169641    608.8875
169642    534.7875
169643    380.8875
169644    483.6875
            ...   
515933    470.0125
515935    478.5000
515936    445.8125
515938    189.0875
515940    487.5750
Name: NOTA_FINAL, Length: 258311, dtype: float64

## Notas de Corte

Para esta etapa é necessário baixar os dados de cortes:
        https://sisu.mec.gov.br/#/relatorio#onepage

Salvar na pasta Data e renomear o arquivo de cada ano para o nome do ano, exceto 2010 a 2018 (que é um arquivo único), renomei para 2010-2018.xsls e salve tbm em .csv

In [113]:
# Abre o arquivo
year_list = ['2023','2022','2021','2020','2019']
media_lic = []
media_med = []
media_eng = []
media_saude = []
media_TI = []
media_DIR = []
for ano in year_list:
    file = f'Data/{ano}.xlsx'
    sheet = f'inscricao_{ano}_1'
    df = pd.read_excel(file, sheet_name=sheet)

    subset1 = df[(df['SG_UF_CAMPUS'] == 'PB') & (df['DS_MOD_CONCORRENCIA'] == 'Ampla concorrência')]

    #Licenciaturas
    subsetlic = subset1[(subset1['DS_GRAU'] == 'Licenciatura')]
    media_lic.append(subsetlic['NU_NOTACORTE'].min())
    
    #Medicina
    subsetmed = subset1[(subset1['NO_CURSO'] == 'MEDICINA')]
    media_med.append(subsetmed['NU_NOTACORTE'].min())
    
    #Outros Saude
    subsetmed = subset1[(subset1['NO_CURSO'] == 'ENFERMAGEM') |
    (subset1['NO_CURSO'] == 'ODONTOLOGIA') | (subset1['NO_CURSO'] == 'FISIOTERAPIA') | 
    (subset1['NO_CURSO'] == 'FONOAUDIOLOGIA')]
    
    media_saude.append(subsetmed['NU_NOTACORTE'].min())
    
    #Engenherias
    subset_eng = subset1[subset1['NO_CURSO'].str.contains('ENGENHARIA', case=False, na=False)]
    media_eng.append(subset_eng['NU_NOTACORTE'].min())

    #TI
    subsetTI = subset1[(subset1['NO_CURSO'] == 'COMPUTAÇÃO')| (subset1['NO_CURSO'] == 'CIÊNCIA DA COMPUTAÇÃO')]
    media_TI.append(subsetTI['NU_NOTACORTE'].min())

    #Direito
    subsetDIR = subset1[(subset1['NO_CURSO'] == 'DIREITO')]
    media_DIR.append(subsetDIR['NU_NOTACORTE'].min())

    

In [114]:
media_med

[np.float64(828.58),
 np.float64(791.4),
 np.float64(795.12),
 np.float64(773.7),
 np.float64(785.02)]

In [115]:
# Abre o arquivo
year_list = ['2011/1','2012/1','2013/1','2014/1','2015/1','2016/1', '2017/1', '2018/1']

file = 'Data/2010_2018.csv'
sheet = 'Sisu 2010 a 2018-Nota de corte'
df = pd.read_csv(file, sep=',', encoding='ISO-8859-1')
df = df[df['NOTA DE CORTE']>0]

for ano in year_list:

    subsetyear = df[(df['EDIÇÃO'] == ano)]
    #Licenciaturas
    subsetlic = subsetyear[(subsetyear['GRAU'] == 'Licenciatura Plena')|(subsetyear['GRAU'] == 'Licenciatura')]
    media_lic.append(subsetlic['NOTA DE CORTE'].min())
    #Medicina
    subsetmed = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'MEDICINA')]
    media_med.append(subsetmed['NOTA DE CORTE'].min())
    #Outros Saude
    subsetmed = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'ENFERMAGEM') |
    (df['NOME CURSO'].str.upper() == 'ODONTOLOGIA') | (df['NOME CURSO'].str.upper() == 'FISIOTERAPIA') | 
    (df['NOME CURSO'].str.upper() == 'FONOAUDIOLOGIA')]
    media_saude.append(subsetmed['NOTA DE CORTE'].min())
    #Engenherias
    subset_eng = subsetyear[subsetyear['NOME CURSO'].str.upper().str.contains('ENGENHARIA', case=False, na=False)]
    media_eng.append(subset_eng['NOTA DE CORTE'].min())
    #TI
    subsetTI = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'COMPUTAÇÃO')| (df['NOME CURSO'].str.upper() == 'CIÊNCIA DA COMPUTAÇÃO')]
    media_TI.append(subsetTI['NOTA DE CORTE'].min())
    #Direito
    subsetDIR = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'DIREITO')]
    media_DIR.append(subsetDIR['NOTA DE CORTE'].min())



C:\Users\maria\AppData\Local\Temp\ipykernel_31536\110422507.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsetmed = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'ENFERMAGEM') |
C:\Users\maria\AppData\Local\Temp\ipykernel_31536\110422507.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsetTI = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'COMPUTAÇÃO')| (df['NOME CURSO'].str.upper() == 'CIÊNCIA DA COMPUTAÇÃO')]
C:\Users\maria\AppData\Local\Temp\ipykernel_31536\110422507.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsetmed = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'ENFERMAGEM') |
C:\Users\maria\AppData\Local\Temp\ipykernel_31536\110422507.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsetTI = subsetyear[(subsetyear['NOME CURSO'].str.upper() == 'COMPUTAÇÃO')| (df['NOME CURSO'].str.upper() == 'CIÊNCIA DA 

In [116]:
# Cria o dataCortes
year_list = list(range(2011, 2024))  # já cria como int
data = [year_list, media_DIR, media_saude, media_eng, media_lic, media_med, media_TI]
dataCortes = pd.DataFrame(data).T
dataCortes.columns = ['NU_ANO', 'Corte_Direito', 'Corte_Saude', 'Corte_Engenharia', 'Corte_Licenciatura', 'Corte_Medicina', 'Corte_TI']

# Faz o merge
dfEnem = dfEnem.merge(dataCortes, on='NU_ANO', how='left')



In [117]:
dfEnem.loc[:, 'aprovacaoSaude'] = np.where(dfEnem['NOTA_FINAL'] >= dfEnem['Corte_Saude'], 1, 0)
dfEnem.loc[:, 'aprovacaoEngenharia'] = np.where(dfEnem['NOTA_FINAL'] >= dfEnem['Corte_Engenharia'], 1, 0)   
dfEnem.loc[:, 'aprovacaoLicenciatura'] = np.where(dfEnem['NOTA_FINAL'] >= dfEnem['Corte_Licenciatura'], 1, 0)
dfEnem.loc[:, 'aprovacaoMedicina'] = np.where(dfEnem['NOTA_FINAL'] >= dfEnem['Corte_Medicina'], 1, 0)
dfEnem.loc[:, 'aprovacaoTI'] = np.where(dfEnem['NOTA_FINAL'] >= dfEnem['Corte_TI'], 1, 0)
dfEnem.loc[:, 'aprovacaoDireito'] = np.where(dfEnem['NOTA_FINAL'] >= dfEnem['Corte_Direito'], 1, 0)

In [118]:
dfEnem = dfEnem.dropna(subset=['NOTA_FINAL'])
dfEnem

,NU_ANO,TP_FAIXA_ETARIA,TP_DEPENDENCIA_ADM_ESC,TP_PRESENCA,NU_NOTA_OBJETIVA,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,...,Corte_Engenharia,Corte_Licenciatura,Corte_Medicina,Corte_TI,aprovacaoSaude,aprovacaoEngenharia,aprovacaoLicenciatura,aprovacaoMedicina,aprovacaoTI,aprovacaoDireito
0,2011,3,2.0,1.0,530.025,160.0,160.0,160.0,160.0,160.0,...,504.00,412.80,828.58,668.37,0,1,1,0,0,0
1,2011,2,4.0,1.0,657.775,140.0,120.0,100.0,140.0,140.0,...,504.00,412.80,828.58,668.37,0,1,1,0,0,0
2,2011,5,2.0,1.0,509.575,120.0,120.0,100.0,120.0,120.0,...,504.00,412.80,828.58,668.37,0,1,1,0,0,0
3,2011,2,2.0,1.0,521.775,80.0,80.0,40.0,80.0,80.0,...,504.00,412.80,828.58,668.37,0,0,0,0,0,0
4,2011,2,4.0,1.0,467.375,100.0,100.0,120.0,100.0,100.0,...,504.00,412.80,828.58,668.37,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258306,2023,3,2.0,1.0,400.025,120.0,120.0,80.0,120.0,120.0,...,589.32,551.84,774.88,587.14,0,0,0,0,0,0
258307,2023,3,4.0,1.0,437.000,120.0,120.0,80.0,120.0,120.0,...,589.32,551.84,774.88,587.14,0,0,0,0,0,0
258308,2023,3,2.0,1.0,431.625,80.0,120.0,100.0,80.0,80.0,...,589.32,551.84,774.88,587.14,0,0,0,0,0,0
258309,2023,2,4.0,1.0,378.175,0.0,0.0,0.0,0.0,0.0,...,589.32,551.84,774.88,587.14,0,0,0,0,0,0


In [119]:
dfEnem.to_csv(f'Data/enem_2011_2023.csv', index=False)